In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from weather import Weather
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("input_data.csv")
keyword = input('Enter keyword: ')
print('Here are some items that match your keyword:')

results = []

for i in range(len(data['Description'])):
    if keyword in data['Description'][i]:
        results.append((i, data['Description'][i]))
    if len(results) == 6:
        break

for result in results:
    print(result[0], ' ', result[1])

print('\n')
print('Please pick an item so we can find more recommendations.')
reference_item = int(input('Enter ID:'))
a = data['Description'][reference_item] + "outfit worn before"
print(a)


Enter keyword: shirt 
Here are some items that match your keyword:
2   Uniqlo navy blue deep crew tshirt: The supima cotton tshirt provides a very elegant casual neat look. Best for comfortable work look. Otherwise wear it with Uniqlo beige denim with black Tommy Hilfiger shoes
21   Banana Republic white ruffles top: T shirt meets ruffles =  comfortable classy trendy white shirt perfect with the grey ankle pants
26   Calvin Klein navy button down satin dress: The shirt dress with golden buttons and zips aspires to give you a royal look
29   Forever 21 red tshirt dress: The most comfortable casual red dress which works like a t shirt. Simple but effective look with brown leather ankle boots
36   Old Navy grey and teal mesh sweatshirt/hoodie: For a casual fun looking sweatshirt in fall. It's warm and comfortable
39   Stanford Summer College Tshirt: a fun reminder of Stanford and a very comfortable large tshirt to wear with good jeans or just for a lazy day at home


Please pick an item s

In [4]:
data = pd.read_csv("input_data.csv")
weather = Weather(unit='c')

current_location = 'Sydney'
location = weather.lookup_by_location(current_location)
condition = location.condition().text()

forecasts = location.forecast()
weather_lo = int(forecasts[0].low())
weather_hi = int(forecasts[0].high())

if weather_hi >= 18 and weather_lo <= 7:
    data['Description'][reference_item] += ' fall spring layer'
    weather_condition = 'Fall/Spring'
elif weather_hi > 20:
    data['Description'][reference_item] += ' hot sunny summer shorts skirt'
    weather_condition = 'Hot'
elif weather_lo < 5:
    data['Description'][reference_item] += ' cold warm cozy sweater'
    weather_condition = 'Cold'
    
message = '{} weather detected in {}. Recommendations are being tailored accordingly.'
print(message.format(weather_condition, current_location))

Hot weather detected in Sydney. Recommendations are being tailored accordingly.


In [5]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['Description'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}

In [6]:
for idx, row in data.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], data['ID'][i]) for i in similar_indices]

    # First item is the item itself, so remove it.
    # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, item_id)
    results[row['ID']] = similar_items[1:]

In [7]:
def item(id):
    return data.loc[data['ID'] == id]['Description'].tolist()[0].split(' - ')[0]

def recommend(item_id, num):
    print("Recommending " + str(num) + " clothes similar to " + item(item_id).split(':')[0] + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print(item(rec[1]) + " (score: " + str(round(rec[0], 5)) + ")")
recommend(item_id=reference_item, num=9)



Recommending 9 clothes similar to Uniqlo navy blue deep crew tshirt...
-------
Uniqlo rayon olive green blouse: This sleeveless cotton blouse is very elegant. It's perfect for dressing up or down for casual or work . Looks best with blue jeans or black denim.Can also be paired with a black skirt. It's perfect for a warm sunny summer day. Black Tommy Hilfiger shoes are perfect with this (score: 0.17178)
Uniqlo blue supima cotton tank: comfortable blue supima cotton tank-top for a minimal look (score: 0.11162)
Uniqlo black supima cotton tank: comfortable black supima cotton tank-top for a minimal look (score: 0.11113)
Uniqlo navy blue and white jumpsuit: A very neat comfortable professional looking jumpsuits. It's perfect for a slightly cold day in Boston in fall to wearing for work.It's great for traveling. It brings out your figure and gives an all-season chic look (score: 0.0832)
Forever 21 maroon crop tshirt: Looks really cute with black shorts or a black skirt on a hot sunny day (sc